In [1]:
!pip -q install --upgrade pip
!pip -q install --upgrade \
        "transformers>=4.40" accelerate bitsandbytes \
        langchain-core langchain-community \
        langchain-huggingface==0.3.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 110.9 MB/s eta 0:00:00


In [17]:
import huggingface_hub
huggingface_hub.login()

In [18]:
import requests, json

DATA_URL = (
    "https://raw.githubusercontent.com/"
    "beefed-up-geek/HCLT-KACL-2025/main/dataset/dev_eng_final.json"
)
dataset = requests.get(DATA_URL, timeout=30).json()

print("Loaded", len(dataset), "records")
print(dataset[0])

Loaded 151 records
{'id': 'nikluge-2025-대화 맥락 추론-dev-000001', 'input': {'conversation': [{'speaker': 1, 'utterance': '안녕하세요! ㅎㅎ', 'utterance_id': 'MDRW2100007084.1.1', 'translated_utterance': 'Hello! Haha'}, {'speaker': 2, 'utterance': '안녕하세요!', 'utterance_id': 'MDRW2100007084.1.2', 'translated_utterance': 'Hello!'}, {'speaker': 2, 'utterance': '동호회활동하세요?', 'utterance_id': 'MDRW2100007084.1.3', 'translated_utterance': 'Are you involved in any club activities?'}, {'speaker': 1, 'utterance': '저는 차 마시는 거 좋아해서 차 좋아하는 사람들이랑 가끔 다회 참석해요', 'utterance_id': 'MDRW2100007084.1.4', 'translated_utterance': 'I enjoy drinking tea, so I sometimes attend tea gatherings with people who also like tea.'}, {'speaker': 1, 'utterance': 'name2님은 동호회활동 참여하는 거 있으신가요?', 'utterance_id': 'MDRW2100007084.1.5', 'translated_utterance': 'Do you participate in any club activities, name2?'}, {'speaker': 2, 'utterance': '오오', 'utterance_id': 'MDRW2100007084.1.6', 'translated_utterance': 'Oh wow!'}, {'speaker': 2, 'utteran

In [19]:
from langchain_huggingface import HuggingFacePipeline, ChatHuggingFace
import torch

MODEL_ID = "meta-llama/Meta-Llama-3.1-8B-Instruct"

llm_pipe = HuggingFacePipeline.from_model_id(
    model_id=MODEL_ID,
    task="text-generation",
    model_kwargs={
        "torch_dtype": torch.bfloat16,
        "device_map": "auto",
        "trust_remote_code": True,
    },
    pipeline_kwargs={
        "max_new_tokens": 512,
        #"temperature": 0,
        #"do_sample": False,
    },
)

chat_model = ChatHuggingFace(llm=llm_pipe)

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

...

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

Device set to use cuda:0


In [20]:
from langchain_core.prompts import ChatPromptTemplate

system_prompt = """
You are an assistant that infers conversational context.
Based on the dialogue below, choose the correct answer.
"""

integrated_prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt.strip()),
    ("human", '''
[Conversation]
{transcript}

[Question]
Among the options below, {question}

[Options]
{inferences}

Do not output anything except A, B, or C.
Return your answer only — no additional text.
''')
])


chain = integrated_prompt | chat_model

In [21]:
def make_transcript(conversation: list[dict]) -> str:
    """화자 1: ..., 화자 2: ... 형식으로 변환"""
    return "\n".join(
        f"speaker {turn['speaker']}: {turn['translated_utterance']}"
        for turn in conversation
    )

def make_inferences(item: dict) -> str:
    """선지를 A/B/C 형식으로 변환"""
    return (
        f"A. {item['inference_translated_1']}\n"
        f"B. {item['inference_translated_2']}\n"
        f"C. {item['inference_translated_3']}"
    )

In [22]:
from tqdm import tqdm
from collections import defaultdict
import pandas as pd
import os
import time
import re

def extract_choice(text: str) -> str:
    """
    LLM 출력에서 A/B/C 중 하나를 찾아 대문자 한 글자로 반환.
    (없으면 빈 문자열 반환)
    1) 정규식으로 단독 토큰 검사 → 빠르고 정확
    2) 없으면 첫 줄 첫 글자를 검사해 파워포인트·리스트·불릿 대응
    """
    # 1)  단독 토큰(앞뒤 공백·개행·문장부호) 패턴
    m = re.search(r'\b([ABC])\b', text.upper())
    if m:
        return m.group(1)

    # 2)  fallback: 첫 줄 첫 글자
    for line in text.splitlines():
        line = line.strip()
        if not line:
            continue
        first = line[0].upper()
        if first in "ABC":
            return first
        break
    return ""

def evaluate(dataset: list[dict], csv_path: str = "eng_llama3_8b.csv") -> None:
    """
    Collect LLM answers (raw + mapped) and append to `csv_path`.
    Adds dialogue metadata and an O/X correctness flag.
    """

    # --- load any existing log ---
    if os.path.exists(csv_path):
        answered_df = pd.read_csv(csv_path)
        answered_ids = set(answered_df["id"])
    else:
        answered_df = pd.DataFrame(
            columns=[
                "id", "transcript", "category", "question", "inferences",
                "ground_truth", "llm_answer_raw", "llm_answer", "is_right"
            ]
        )
        answered_ids = set()

    # --- main loop ---
    buffer = []
    for record in tqdm(dataset, desc="Evaluating", unit="item"):
        if record["id"] in answered_ids:
            continue

        inp = record["input"]
        transcript_str = make_transcript(inp["conversation"])
        inferences_str = make_inferences(inp)

        prompt_vars = {
            "transcript": transcript_str,
            "question":  inp["question"],
            "inferences": inferences_str,
        }

        # retry on rate-limit (429) by sleeping 61 s
        while True:
            try:
                answer_raw = chain.invoke(prompt_vars).content.strip()  # "A"|"B"|"C"
                break
            except Exception as err:
                if "429" in str(err) or "rate exceeded" in str(err).lower():
                    print("[RateLimit] 429 encountered — sleeping 61 s …")
                    time.sleep(61)
                    continue
                print(f"[ERROR] {err!r} — stopping early, progress saved.")
                if buffer:
                    pd.DataFrame(buffer).to_csv(
                        csv_path,
                        mode="a",
                        header=not os.path.exists(csv_path),
                        index=False,
                        encoding="utf-8-sig",
                    )
                return  # abort evaluation

        # map A/B/C → inference key
        letter = extract_choice(answer_raw)
        letter_to_key = {"A": "inference_1", "B": "inference_2", "C": "inference_3"}
        key = letter_to_key.get(letter, "")
        is_right = "O" if key == record["output"] else "X"

        buffer.append({
            "id": record["id"],
            "transcript": transcript_str,
            "category": inp["category"],
            "question": inp["question"],
            "inferences": inferences_str,
            "ground_truth": record["output"],
            "llm_answer_raw": letter,
            "llm_answer": key,
            "is_right": is_right,
        })

        # flush every 10 rows
        if len(buffer) >= 10:
            pd.DataFrame(buffer).to_csv(
                csv_path,
                mode="a",
                header=not os.path.exists(csv_path),
                index=False,
                encoding="utf-8-sig",
            )
            buffer.clear()

    # flush any leftover rows
    if buffer:
        pd.DataFrame(buffer).to_csv(
            csv_path,
            mode="a",
            header=not os.path.exists(csv_path),
            index=False,
            encoding="utf-8-sig",
        )

def print_results(csv_path: str = "eng_llama3_8b.csv") -> None:
    """Read `csv_path` and display accuracy by category + overall."""
    if not os.path.exists(csv_path):
        print("No results file found — run evaluate() first.")
        return

    df = pd.read_csv(csv_path)
    if df.empty:
        print("Results file is empty.")
        return

    if "is_right" in df.columns:
        df["correct"] = df["is_right"] == "O"
    else:
        df["correct"] = df["ground_truth"] == df["llm_answer"]

    print("=== 문제 유형별 정답률 ===")
    for cat, group in df.groupby("category"):
        corr = group["correct"].sum()
        tot = len(group)
        print(f"{cat}: {corr}/{tot} = {corr / tot:.2%}")

    overall_corr = df["correct"].sum()
    overall_tot = len(df)
    print("\n=== 전체 정답률 ===")
    print(f"{overall_corr}/{overall_tot} = {overall_corr / overall_tot:.2%}")


In [23]:
evaluate(dataset)

Evaluating: 100%|██████████| 151/151 [00:16<00:00,  9.15item/s]


In [24]:
print_results()

=== 문제 유형별 정답률 ===
동기: 8/25 = 32.00%
반응: 9/25 = 36.00%
원인: 7/25 = 28.00%
전제: 5/25 = 20.00%
후행사건: 19/51 = 37.25%

=== 전체 정답률 ===
48/151 = 31.79%


In [16]:
! nvidia-smi

Tue Jul 22 08:03:05 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off |   00000000:00:04.0 Off |                    0 |
| N/A   33C    P0             54W /  400W |   16157MiB /  40960MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----